# Atividade Prática: Construindo Endpoints de uma API ML

**Objetivo:** Construir os 4 endpoints da API FastAPI para classificação de vinhos

**Arquivo de trabalho:** `api/main_template.py`


---

## Importante!

- Leia cada instrução com atenção
- Pesquise na documentação quando necessário
- Teste seu código frequentemente

---

## PREPARAÇÃO

### 1. Treinar o modelo

```bash
python scripts/train_model.py
```

### 2. Verificar se o modelo foi criado

```bash
ls -la models/wine_model.pkl
```

**Checkpoint:** O arquivo `wine_model.pkl` existe?

---

## ENTENDENDO O PROJETO

### Abra o arquivo `api/main_template.py`

Observe que o arquivo possui várias seções:

1. **Importações** - Já prontas
2. **CRIAR APP** - Já pronto
3. **MODELOS DE DADOS** - Já prontos
4. **VARIÁVEIS GLOBAIS** - Já prontas
5. **CARREGAR MODELO** - Já pronto
6. **ENDPOINTS** - VOCÊ VAI COMPLETAR AQUI!

### Sua missão:

Completar os 4 TODOs na seção de ENDPOINTS:

- PASSO 1: `GET /`
- PASSO 2: `GET /health`
- PASSO 3: `POST /predict`
- PASSO 4: `GET /stats`

---

## CONCEITOS IMPORTANTES

Antes de começar, você precisa entender:

### 1. Decorators em FastAPI

FastAPI usa decorators para definir rotas:

```python
@app.get("/caminho")      # Define uma rota GET
@app.post("/caminho")     # Define uma rota POST
```

### 2. Métodos HTTP

- **GET** - Buscar dados (ex: ver informações)
- **POST** - Enviar dados (ex: fazer predição)

### 3. Retorno de Dados

FastAPI converte automaticamente dicionários Python para JSON:

```python
return {"chave": "valor"}  # Vira {"chave": "valor"} em JSON
```

### 4. Pydantic Models

Já estão definidos no template:
- `WineInput` - Define os dados de entrada (11 características do vinho)
- `PredictionOutput` - Define o formato da resposta

---

## PASSO 1: Endpoint Raiz - GET /

**Objetivo:** Criar uma página inicial que mostra informações sobre a API.

### O que fazer:

1. Localize o `TODO: PASSO 1` no arquivo `api/main_template.py`
2. Descomente as linhas
3. Complete a função `home()`

### Requisitos:

A função deve retornar um dicionário contendo:
- `"message"` - Uma mensagem de boas-vindas
- `"version"` - A versão da API (1.0.0)
- `"endpoints"` - Um dicionário com os endpoints disponíveis:
  - `"docs"` → `"/docs"`
  - `"health"` → `"/health"`
  - `"predict"` → `"/predict (POST)"`

### Perguntas para refletir:

1. Por que usar `@app.get("/")` e não `@app.post("/")`?
2. O que acontece quando alguém acessa `http://localhost:8000/`?
3. Por que retornar um dicionário em vez de uma string?

### Recursos:

- Documentação FastAPI: https://fastapi.tiangolo.com/tutorial/first-steps/
- Busque por "FastAPI first endpoint"

### Teste:

```bash
uvicorn api.main_template:app --reload
```

Abra: http://localhost:8000/

**Checkpoint:** Você consegue ver as informações da API no navegador?

---

## PASSO 2: Endpoint Health Check - GET /health

**Objetivo:** Criar um endpoint para monitorar o status da API.

### O que fazer:

1. Localize o `TODO: PASSO 2` no arquivo
2. Descomente as linhas
3. Complete a função `health_check()`

### Requisitos:

A função deve retornar um dicionário contendo:
- `"status"` - String indicando que está "online"
- `"model_loaded"` - Valor da variável global `model_loaded`
- `"total_predictions"` - Valor da variável global `total_predictions`
- `"timestamp"` - Timestamp atual no formato ISO

### Dicas:

- As variáveis globais já estão definidas no topo do arquivo
- Para timestamp ISO, use: `datetime.now().isoformat()`
- A biblioteca `datetime` já está importada

### Perguntas para refletir:

1. Por que é importante ter um endpoint de health check?
2. O que `model_loaded` indica?
3. Como sistemas de monitoramento usariam este endpoint?

### Teste:

```bash
curl http://localhost:8000/health
```

**Checkpoint:** O `model_loaded` está como `true`?

---

## TESTE INTERMEDIÁRIO

Antes de continuar para o endpoint mais complexo, vamos testar o que você construiu:

### 1. Swagger UI

Abra: http://localhost:8000/docs

Você deve ver:
- Documentação automática da API
- Os 2 endpoints que você criou
- Possibilidade de testar cada um

### 2. Teste cada endpoint no Swagger

### 3. Perguntas:

1. Como o Swagger sabe quais endpoints existem?
2. Por que a documentação é gerada automaticamente?
3. Qual a vantagem de ter essa interface?

**Checkpoint:** Ambos os endpoints estão funcionando? Se sim, Ctrl+C e continue!

---

## PASSO 3: Endpoint de Predição - POST /predict

**ATENÇÃO:** Este é o endpoint mais importante e complexo!


Este passo está dividido em várias partes. Leia TODAS antes de começar!

---

## PASSO 3.1: Estrutura Básica do Endpoint

### Requisitos:

1. **Decorator:** `@app.post("/predict", response_model=PredictionOutput)`
2. **Parâmetro:** `wine: WineInput`
3. **Variável global:** `global total_predictions`

### Perguntas:

1. Por que POST e não GET?
2. O que significa `response_model=PredictionOutput`?
3. Por que o parâmetro tem tipo `WineInput`?

---

## PASSO 3.2: Verificação do Modelo

### Tarefa:

Antes de fazer qualquer predição, você precisa verificar se o modelo está carregado.

### O que fazer:

1. Verifique a variável global `model_loaded`
2. Se for `False`, lance uma exceção HTTP

### Como lançar exceção HTTP:

```python
raise HTTPException(
    status_code=503,  # Service Unavailable
    detail="Mensagem de erro aqui"
)
```

### Perguntas:

1. O que significa o código HTTP 503?
2. Por que não simplesmente retornar `None`?

### Pesquise:

- "HTTP status codes"
- "FastAPI HTTPException"

---

## PASSO 3.3: Preparar os Dados

### Desafio:

O modelo ML espera receber os dados em um formato específico:
- **Tipo:** Numpy array
- **Shape:** `(1, 11)` - 1 amostra com 11 características
- **Ordem:** As 11 características na ordem correta

### Campos do WineInput (NA ORDEM):

1. `fixed_acidity`
2. `volatile_acidity`
3. `citric_acid`
4. `residual_sugar`
5. `chlorides`
6. `free_sulfur_dioxide`
7. `total_sulfur_dioxide`
8. `density`
9. `pH`
10. `sulphates`
11. `alcohol`

### Dica:

Use `np.array([[ ... ]])` para criar um array 2D.

### Perguntas:

1. Por que precisa ser um array 2D `[[...]]` e não 1D `[...]`?
2. Por que a ordem importa?

---

## PASSO 3.4: Fazer a Predição

### Tarefa:

Use o modelo carregado para fazer a predição.

### O que fazer:

1. Use `model.predict(features)` para obter a classe prevista
2. Use `model.predict_proba(features)` para obter as probabilidades
3. Pegue o primeiro elemento de cada resultado (índice `[0]`)

### Perguntas:

1. O que `predict()` retorna?
2. O que `predict_proba()` retorna?
3. Por que pegar `[0]`?
4. Qual a diferença entre os dois métodos?

### Pesquise:

- "sklearn predict vs predict_proba"
- "classification probabilities"

---

## PASSO 3.5: Converter e Calcular

### Tarefa 1: Converter ID para Nome

O modelo retorna números (0, 1, 2), mas queremos nomes legíveis.

**Mapeamento:**
- 0 → "Ruim"
- 1 → "Médio"
- 2 → "Bom"

**Como fazer:** Use um dicionário.

---

### Tarefa 2: Calcular Confiança

A confiança é a maior probabilidade entre todas as classes.

**Dica:** Use `max()` e converta para `float()`.

**Perguntas:**
1. Por que usar `float()`?
2. O que significa uma confiança de 0.9?

---

### Tarefa 3: Criar Dicionário de Probabilidades

Queremos mostrar a probabilidade de cada classe.

**Formato esperado:**
```python
{
    "Ruim": 0.1,
    "Médio": 0.2,
    "Bom": 0.7
}
```

**Perguntas:**
1. A soma das probabilidades deve dar quanto?
2. Por que mostrar todas as probabilidades?

---

## PASSO 3.6: Retornar o Resultado

### Tarefa 1: Incrementar Contador

Incremente a variável global `total_predictions`.

**Pergunta:** Por que fazer isso?

---

### Tarefa 2: Retornar PredictionOutput

Use o modelo Pydantic já definido no arquivo.

**Campos necessários:**
- `quality` - Nome da classe
- `confidence` - Maior probabilidade
- `probabilities` - Dicionário com todas as probabilidades
- `timestamp` - Use `datetime.now().isoformat()`

**Perguntas:**
1. Por que usar `PredictionOutput` em vez de um dicionário simples?
2. O que acontece se faltarem campos?

---

## PASSO 3.7: Tratamento de Erros

### Desafio:

Todo o código de predição deve estar dentro de um bloco `try-except`.

### Como fazer:

```python
try:
    # Todo o código de predição aqui
    
except Exception as e:
    raise HTTPException(
        status_code=500,
        detail=f"Erro ao fazer predição: {str(e)}"
    )
```

### Perguntas:

1. O que significa o código HTTP 500?
2. Por que capturar exceções?
3. Que tipos de erros podem ocorrer?

---

## PASSO 3.8: Checklist do Endpoint /predict

Antes de testar, verifique:

- [ ] Decorator correto com `response_model`
- [ ] Declaração de `global total_predictions`
- [ ] Verificação de `model_loaded`
- [ ] Bloco `try-except`
- [ ] Array numpy com 11 características
- [ ] Chamadas de `predict()` e `predict_proba()`
- [ ] Conversão de ID para nome
- [ ] Cálculo de confiança
- [ ] Dicionário de probabilidades
- [ ] Incremento do contador
- [ ] Retorno de `PredictionOutput`

**Checkpoint:** Todos os itens estão implementados?

---

## PASSO 4: Endpoint de Estatísticas - GET /stats

### O que fazer:

1. Localize o `TODO: PASSO 4` no arquivo
2. Descomente as linhas
3. Complete a função `get_stats()`

### Requisitos:

A função deve retornar um dicionário contendo:
- `"total_predictions"` - Valor da variável global
- `"model_loaded"` - Valor da variável global
- `"status"` - String "active"

### Perguntas:

1. Qual a diferença entre `/health` e `/stats`?
2. Quando usar um ou outro?

**Checkpoint:** O `total_predictions` incrementa após cada predição?

---

## TESTE FINAL COMPLETO

### 1. Iniciar a API

```bash
uvicorn api.main_template:app --reload
```

### 2. Verificar console

Deve aparecer:
```
Modelo carregado com sucesso!
INFO:     Uvicorn running on http://127.0.0.1:8000
```

### 3. Swagger UI

Abra: http://localhost:8000/docs

Teste cada endpoint:
- GET /
- GET /health
- POST /predict (use o exemplo que aparece)
- GET /stats

### 4. Via curl

```bash
# Predição de vinho BOM
curl -X POST http://localhost:8000/predict \
  -H "Content-Type: application/json" \
  -d '{"fixed_acidity": 7.0, "volatile_acidity": 0.3, "citric_acid": 0.5, "residual_sugar": 2.0, "chlorides": 0.05, "free_sulfur_dioxide": 30.0, "total_sulfur_dioxide": 100.0, "density": 0.995, "pH": 3.2, "sulphates": 0.6, "alcohol": 13.0}'
```

**Checkpoint:** Funcionou?

---

## VALIDAÇÃO FINAL

### Checklist de Código:

- [ ] Endpoint `GET /` implementado e funcionando
- [ ] Endpoint `GET /health` implementado e funcionando
- [ ] Endpoint `POST /predict` implementado e funcionando
- [ ] Endpoint `GET /stats` implementado e funcionando
- [ ] Swagger UI acessível e documentado
- [ ] Predições retornam formato correto
- [ ] Contador incrementa corretamente

### Checklist de Entendimento:

- [ ] Entendo a diferença entre GET e POST
- [ ] Entendo como funcionam os decorators do FastAPI
- [ ] Entendo o papel do Pydantic na validação
- [ ] Entendo como preparar dados para o modelo
- [ ] Entendo `predict()` vs `predict_proba()`
- [ ] Entendo códigos de status HTTP
- [ ] Entendo tratamento de erros
- [ ] Entendo o fluxo completo de predição

---

## RECAPITULANDO

### O que você aprendeu?

**APIs REST:**
- Endpoints e rotas
- Métodos HTTP (GET, POST)
- Códigos de status
- Request/Response

**FastAPI:**
- Decorators para rotas
- Validação automática com Pydantic
- Documentação automática (Swagger)
- Tratamento de erros

**Machine Learning:**
- Carregar modelo treinado
- Preparar dados para inferência
- Fazer predições
- Interpretar probabilidades

**MLOps:**
- Health checks
- Monitoramento básico
- Métricas
- Model serving

---

### Perguntas finais para refletir:

1. Como você melhoraria esta API?
2. Que problemas podem ocorrer em produção?
3. Como você monitoraria esta API?
4. Como você escalaria esta API?
5. Que segurança você adicionaria?

---
